<p><span style="font-size:36px"><strong>1 Data Collection</strong></span></p>

In [ ]:
import pandas as pd
import geojson as geojs
import json as js
import requests

<p><span style="font-size:18px"><strong>1.1 Gemeindegrenzen von geo.admin.ch sammeln</strong></span></p>

Da ein kompletter, aktueller Datensatz aller schweizer Gemeindegrenzen nicht ausfindig gemacht werden konnte, wurden die aktuellen Grenzen aller Gemeinden einzeln von der GEO.Admin API heruntergeladen. Weil dafür die BFS ID der einzelnen Gemeinden nötig ist um die Gemeindegrenzen der API anzufragen, jedoch nach einiger Zeit nicht gefunden werden konnte und es ersichtlich war das die ID's nur bis 7000 zählen, wird bei der Anfrage jede ID von 0 bis 7000 geprüft, so dass alle Gemeinden sicher erfasst werden.

In [ ]:
#Lädt die Gemeindegrenzen von der Geo.Admin API herunter und speichert die Daten in Listen
#ACHTUNG: Das Herunterladen der Daten braucht mind. eine halbe Stunde
BBoxCoord =[]
BFSid =[]
MultiploygonCoord =[]
GemeindeName =[]
GemeindeKanton =[]
GemeindeFläche =[]
GemeindeLabel =[]

for x in range(0, 7000):
    x = requests.get(f'https://api3.geo.admin.ch/rest/services/api/MapServer/ch.swisstopo.swissboundaries3d-gemeinde-flaeche.fill/{x}?geometryFormat=geojson&sr=4326')
    try:
        gjs = geojs.loads(x.text)
        BBoxCoord.append(gjs['feature']['bbox'])
        BFSid.append(gjs['feature']['featureId'])
        MultiploygonCoord.append(gjs['feature']['geometry'])
        GemeindeName.append(gjs['feature']["properties"]['gemname'])
        GemeindeKanton.append(gjs['feature']["properties"]['kanton'])
        GemeindeFläche.append(gjs['feature']["properties"]['gemflaeche'])
        GemeindeLabel.append(gjs['feature']["properties"]["label"])
        print(gjs['feature']["properties"]["label"], gjs['feature']['featureId'])
        print(x.status_code)
    except:
        print('error')
        print(x.status_code)

In [ ]:
#Listen werden zu einem DataFrame zusammengefügt
f = {'BFSid':BFSid, 'GemeindeLabel':GemeindeLabel, 'GemeindeName':GemeindeName, 'GemeindeKanton':GemeindeKanton, 'GemeindeFläche':GemeindeFläche, 'Coord BBox':BBoxCoord, 'Polygon':MultiploygonCoord}
gemdf = pd.DataFrame(data=f)

In [ ]:
#Kontrolle
gemdf

In [ ]:
#DataFrame wird für weitere Verwendung abgespeichert
gemdf.to_csv('../Data/GemeindeGrenzen.csv')